In [2]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [11]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432")

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
f = 1.0                                            # desired frac
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT content, type_id FROM fakenews.article", connection).sample(frac=f, random_state=seed)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df

,content,type_id
4300,submitted by nick giambruno via URL obama pull...,3
9536,raw content limited official use page NUM bogo...,7
813,russia and china stand in agreement on ukraine...,3
11248,who ever thought that torture could be so funn...,8
1436,america is changing the civil war is over and ...,1
...,...,...
10955,national review at NUM onward by the editors —...,8
905,create your own logo with logo marker online t...,0
5192,most people understand that we live in a pollu...,4
12172,he was asking me to take the baby with me out ...,9


In [12]:
pd.read_sql_query("SELECT * FROM fakenews.type", connection).sample(frac=f, random_state=seed)

,type_id,type
2,2,bias
9,9,reliable
6,6,clickbait
4,4,junksci
0,0,fake
3,3,conspiracy
1,1,satire
7,7,unreliable
8,8,political
5,5,hate


In [13]:
# to reliable
df.type_id = df.type_id.replace (2, 9).values
df.type_id = df.type_id.replace (6, 9)
df.type_id = df.type_id.replace (8, 9)

# to fake
df.type_id = df.type_id.replace (1, 0)
df.type_id = df.type_id.replace (3, 0)
df.type_id = df.type_id.replace (4, 0)
df.type_id = df.type_id.replace (5, 0)
df.type_id = df.type_id.replace (7, 0)

df.head()

,content,type_id
4300,submitted by nick giambruno via URL obama pull...,0
9536,raw content limited official use page NUM bogo...,0
813,russia and china stand in agreement on ukraine...,0
11248,who ever thought that torture could be so funn...,9
1436,america is changing the civil war is over and ...,0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

x = v.fit_transform(df['content'])

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, df["type_id"], test_size=0.2, random_state=0)

In [17]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(11, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [18]:
model.fit(X_train.toarray(), y_train.values, epochs=3)

Train on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.8505 - accuracy: 0.6600
Epoch 2/3
10000/10000 [==============================] - 16s 2ms/sample - loss: 0.2914 - accuracy: 0.8941
Epoch 3/3
10000/10000 [==============================] - 16s 2ms/sample - loss: 0.0929 - accuracy: 0.9819


In [19]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test.toarray(),y_test.values)
print(val_loss, val_acc)

2500/2500 [==============================] - 4s 2ms/sample - loss: 0.3894 - accuracy: 0.8152
0.3894043992578983 0.8152
